In [1]:
import pandas as pd

In [2]:
diagnoses = pd.read_csv('/Users/grace/mimic/csv/DIAGNOSES_ICD.csv', sep=',')
diagnoses.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


### Filter data by sepsis diagnosis

In [3]:
target_diag  = diagnoses[diagnoses.ICD9_CODE.isin(['77181', '99591', '99592', '67020', '67022', '67024'])]

In [4]:
target_diag.shape

(5409, 5)

### Total number of patients

In [5]:
target_diag.count()

ROW_ID        5409
SUBJECT_ID    5409
HADM_ID       5409
SEQ_NUM       5409
ICD9_CODE     5409
dtype: int64

In [6]:
len(target_diag.SUBJECT_ID.unique())

4781

In [7]:
target_diag.groupby(target_diag.SUBJECT_ID)#['ICD9_CODE'].count()#.sort('ICD9_CODE').index[-1]

In [8]:
target_diag.groupby(target_diag.SUBJECT_ID).count()[['ICD9_CODE']].sort_values(by=['ICD9_CODE'], ascending=False)[:10]

,ICD9_CODE
SUBJECT_ID,
41976,12
4787,8
3977,7
3866,7
73713,7
29426,6
49623,6
19029,5
9555,5


In [9]:
labs = pd.read_csv('/Users/grace/mimic/csv/LABEVENTS.csv', sep=',')
labs.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [10]:
lab_items = pd.read_csv('/Users/grace/mimic/csv/D_LABITEMS.csv', sep=',')
lab_items.head()

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN


In [11]:
lab_items[lab_items.LABEL.str.find('Vancomycin')>=0]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
335,209,51009,Vancomycin,Blood,Chemistry,20578-1


In [12]:
va_labs = labs[(labs.ITEMID==51009) & (labs.SUBJECT_ID.isin(target_diag.SUBJECT_ID.unique()))]
va_labs.groupby(va_labs.SUBJECT_ID).count()[['CHARTTIME']].sort_values(by=['CHARTTIME'], ascending=False)[:10]

,CHARTTIME
SUBJECT_ID,
13033,138
18353,114
25030,110
19620,104
433,98
11923,97
22815,84
20124,81
16655,76


In [13]:
target_diag[target_diag.SUBJECT_ID==25030]

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
279041,279660,25030,102205,10.0,99592
279050,279669,25030,128473,8.0,99592
279058,279677,25030,132107,3.0,99592


### Patient Subject_ID = 25030

In [14]:
#diagnosis
diagnoses[diagnoses.SUBJECT_ID==25030].shape

(53, 5)

In [15]:
unique_d_for_25030 = diagnoses[diagnoses.SUBJECT_ID==25030]['ICD9_CODE'].unique()
unique_d_for_25030

array(['0389', '4275', '78551', '4260', '4210', '40391', '25041', '2767',
       'V4975', '99592', '41092', '99662', '03842', '2869', '99681',
       '4254', '2851', 'V5867', '51889', '7904', '71941', '0383', '70703',
       '78552', '07070', '28521', '40301', '43491', '4372', '2720',
       '25051', '2761', '36201', '431', '3314', '7070', '25081', '78039',
       '51881'], dtype=object)

In [40]:
dia_items = pd.read_csv('/Users/grace/mimic/csv/D_ICD_DIAGNOSES.csv', sep=',')
dia_items.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [45]:
dia_items[dia_items.ICD9_CODE=='7070']

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE


In [46]:
sep_diag = dia_items[dia_items.ICD9_CODE.isin(unique_d_for_25030)][['ICD9_CODE', 'SHORT_TITLE']].set_index('ICD9_CODE').to_dict()
sep_diag = sep_diag['SHORT_TITLE']
sep_diag['7070'] = ''
sep_diag

{'0383': 'Anaerobic septicemia',
 '03842': 'E coli septicemia',
 '0389': 'Septicemia NOS',
 '07070': 'Hpt C w/o hepat coma NOS',
 '25041': 'DMI renl nt st uncntrld',
 '25051': 'DMI ophth nt st uncntrld',
 '25081': 'DMI oth nt st uncntrld',
 '2720': 'Pure hypercholesterolem',
 '2761': 'Hyposmolality',
 '2767': 'Hyperpotassemia',
 '2851': 'Ac posthemorrhag anemia',
 '28521': 'Anemia in chr kidney dis',
 '2869': 'Coagulat defect NEC/NOS',
 '3314': 'Obstructiv hydrocephalus',
 '36201': 'Diabetic retinopathy NOS',
 '40301': 'Mal hyp kid w cr kid V',
 '40391': 'Hyp kid NOS w cr kid V',
 '41092': 'AMI NOS, subsequent',
 '4210': 'Ac/subac bact endocard',
 '4254': 'Prim cardiomyopathy NEC',
 '4260': 'Atriovent block complete',
 '4275': 'Cardiac arrest',
 '431': 'Intracerebral hemorrhage',
 '43491': 'Crbl art ocl NOS w infrc',
 '4372': 'Hypertens encephalopathy',
 '51881': 'Acute respiratry failure',
 '51889': 'Other lung disease NEC',
 '7070': '',
 '70703': 'Pressure ulcer, low back',
 '71941':

In [48]:
dia_all_25030.ICD9_CODE.unique()

array(['0389', '4275', '78551', '4260', '4210', '40391', '25041', '2767',
       'V4975', '99592', '41092', '99662', '03842', '2869', '99681',
       '4254', '2851', 'V5867', '51889', '7904', '71941', '0383', '70703',
       '78552', '07070', '28521', '40301', '43491', '4372', '2720',
       '25051', '2761', '36201', '431', '3314', '7070', '25081', '78039',
       '51881'], dtype=object)

In [50]:
dia_all_25030['SHORT_TITLE'] = dia_all_25030['ICD9_CODE'].apply(lambda x: sep_diag[x])
dia_all_25030

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,SHORT_TITLE
279032,279651,25030,102205,1.0,0389,Septicemia NOS
279033,279652,25030,102205,2.0,4275,Cardiac arrest
279034,279653,25030,102205,3.0,78551,Cardiogenic shock
279035,279654,25030,102205,4.0,4260,Atriovent block complete
279036,279655,25030,102205,5.0,4210,Ac/subac bact endocard
279037,279656,25030,102205,6.0,40391,Hyp kid NOS w cr kid V
279038,279657,25030,102205,7.0,25041,DMI renl nt st uncntrld
279039,279658,25030,102205,8.0,2767,Hyperpotassemia
279040,279659,25030,102205,9.0,V4975,Status amput below knee
279041,279660,25030,102205,10.0,99592,Severe sepsis


## lab

In [16]:
unique_lab_for_25030 = labs[labs.SUBJECT_ID==25030]['ITEMID'].unique()
unique_lab_for_25030

array([51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277,
       51279, 51301, 50979, 51114, 51116, 51120, 51125, 51127, 51128,
       50862, 50868, 50882, 50893, 50902, 50912, 50931, 50960, 50970,
       50971, 50983, 51006, 51221, 50909, 50954, 50800, 50802, 50804,
       50812, 50813, 50817, 50818, 50820, 50821, 50825, 50910, 50911,
       51003, 51009, 50809, 50822, 50861, 50863, 50867, 50878, 50885,
       50933, 50956, 51137, 51144, 51146, 51151, 51200, 51233, 51244,
       51246, 51252, 51254, 51256, 51267, 51268, 50967, 51008, 50816,
       50828, 50965, 50829, 50831, 51143, 51251, 51255, 51294, 50852,
       50903, 50904, 50905, 50907, 51000, 50887, 50803, 50976, 51266,
       51278, 50908, 51002, 50857, 51014, 50927, 51260, 50806, 50808,
       50824, 51018, 51351, 51355, 51360, 51362, 51363, 51043, 51427,
       51431, 51436, 51438, 51439, 50819, 50826, 50886, 50919, 51257,
       50924, 50952, 50953, 50998, 50929, 50993, 50810, 50811, 51386,
       50968, 50969,

In [17]:
lab_items[lab_items.ITEMID.isin(unique_lab_for_25030)]

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
5,551,51351,Lymphs,Cerebrospinal Fluid (CSF),Hematology,26479-6
6,552,51352,Macrophage,Cerebrospinal Fluid (CSF),Hematology,30426-1
9,555,51355,Monocytes,Cerebrospinal Fluid (CSF),Hematology,26486-1
14,560,51360,Polys,Cerebrospinal Fluid (CSF),Hematology,26517-3
16,562,51362,"RBC, CSF",Cerebrospinal Fluid (CSF),Hematology,26454-9
17,563,51363,"WBC, CSF",Cerebrospinal Fluid (CSF),Hematology,26465-5
40,586,51386,Bands,Other Body Fluid,Hematology,26510-8
73,619,51419,Eosinophils,Other Body Fluid,Hematology,26452-3
78,624,51424,Immunophenotyping,Other Body Fluid,Hematology,NaN


In [27]:
labels = lab_items[lab_items.ITEMID.isin(unique_lab_for_25030)][['LABEL', 'ITEMID']].set_index('ITEMID').to_dict()
labels = labels['LABEL']
labels

{50800: 'SPECIMEN TYPE',
 50802: 'Base Excess',
 50803: 'Calculated Bicarbonate, Whole Blood',
 50804: 'Calculated Total CO2',
 50805: 'Carboxyhemoglobin',
 50806: 'Chloride, Whole Blood',
 50808: 'Free Calcium',
 50809: 'Glucose',
 50810: 'Hematocrit, Calculated',
 50811: 'Hemoglobin',
 50812: 'Intubated',
 50813: 'Lactate',
 50814: 'Methemoglobin',
 50815: 'O2 Flow',
 50816: 'Oxygen',
 50817: 'Oxygen Saturation',
 50818: 'pCO2',
 50819: 'PEEP',
 50820: 'pH',
 50821: 'pO2',
 50822: 'Potassium, Whole Blood',
 50824: 'Sodium, Whole Blood',
 50825: 'Temperature',
 50826: 'Tidal Volume',
 50827: 'Ventilation Rate',
 50828: 'Ventilator',
 50829: 'Fluid Type',
 50831: 'pH',
 50849: 'Total Protein, Ascites',
 50852: '% Hemoglobin A1c',
 50856: 'Acetaminophen',
 50857: 'Acetone',
 50861: 'Alanine Aminotransferase (ALT)',
 50862: 'Albumin',
 50863: 'Alkaline Phosphatase',
 50867: 'Amylase',
 50868: 'Anion Gap',
 50871: 'Anti-Mitochondrial Antibody',
 50873: 'Anti-Nuclear Antibody',
 50878: 'As

In [29]:
lab_all_25030 = labs[labs.SUBJECT_ID==25030]
lab_all_25030.shape

(6828, 9)

In [28]:
lab_all_25030['LABEL'] = lab_all_25030.ITEMID.apply(lambda x: labels[x])

## procedure

In [18]:
proc = pd.read_csv('/Users/grace/mimic/csv/PROCEDURES_ICD.csv', sep=',')
proc.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [19]:
unique_proc_for_25030 = proc[proc.SUBJECT_ID==25030]['ICD9_CODE'].unique()
unique_proc_for_25030

array([5498, 3893, 3895,   17, 9904, 3995,  311,   22,  234, 4011, 4432,
        966, 9915,  331, 9672, 9604, 9969, 9907, 8604, 8872])

In [20]:
proc_items = pd.read_csv('/Users/grace/mimic/csv/D_ICD_PROCEDURES.csv', sep=',')
proc_items.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,264,851,Canthotomy,Canthotomy
1,265,852,Blepharorrhaphy,Blepharorrhaphy
2,266,859,Adjust lid position NEC,Other adjustment of lid position
3,267,861,Lid reconst w skin graft,Reconstruction of eyelid with skin flap or graft
4,268,862,Lid reconst w muc graft,Reconstruction of eyelid with mucous membrane ...


In [21]:
proc_items[proc_items.ICD9_CODE.isin(unique_proc_for_25030)]

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
336,148,17,Infusion of vasopressor,Infusion of vasopressor agent
340,152,22,IVUS intrathoracic ves,Intravascular imaging of intrathoracic vessels
375,187,331,Spinal tap,Spinal tap
421,233,234,Ventricl shunt-abdomen,Ventricular shunt to abdominal cavity and organs
453,552,311,Temporary tracheostomy,Temporary tracheostomy
505,604,331,Incision of lung,Incision of lung
702,801,3893,Venous cath NEC,"Venous catheterization, not elsewhere classified"
704,803,3895,Ven cath renal dialysis,Venous catheterization for renal dialysis
1586,1291,5498,Peritoneal dialysis,Peritoneal dialysis
1660,1365,4432,Percu gastrojejunostomy,Percutaneous [endoscopic] gastrojejunostomy


In [22]:
short_titles = proc_items[proc_items.ICD9_CODE.isin(unique_proc_for_25030)][['ICD9_CODE', 'SHORT_TITLE']].set_index('ICD9_CODE').to_dict()
short_titles = short_titles['SHORT_TITLE']
short_titles

{17: 'Infusion of vasopressor',
 22: 'IVUS intrathoracic ves',
 234: 'Ventricl shunt-abdomen',
 311: 'Temporary tracheostomy',
 331: 'Incision of lung',
 966: 'Entral infus nutrit sub',
 3893: 'Venous cath NEC',
 3895: 'Ven cath renal dialysis',
 3995: 'Hemodialysis',
 4011: 'Lymphatic struct biopsy',
 4432: 'Percu gastrojejunostomy',
 5498: 'Peritoneal dialysis',
 8604: 'Other skin & subq i & d',
 8872: 'Dx ultrasound-heart',
 9604: 'Insert endotracheal tube',
 9672: 'Cont inv mec ven 96+ hrs',
 9904: 'Packed cell transfusion',
 9907: 'Serum transfusion NEC',
 9915: 'Parent infus nutrit sub',
 9969: 'Cardiac rhythm conv NEC'}

In [23]:
proc_all_25030 = proc[proc.SUBJECT_ID==25030]
proc_all_25030.shape

(31, 5)

In [25]:
proc_all_25030['SHORT_TITLE'] = proc[proc.SUBJECT_ID==25030].ICD9_CODE.apply(lambda x: short_titles[x])
proc_all_25030.to_csv('./proc.csv')

/Users/grace/workspace/keras/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### Patient General Information

In [ ]:
admission = pd.read_csv('/Users/grace/mimic/csv/ADMISSIONS.csv', sep=',')
admission.head()

In [ ]:
admission[admission.SUBJECT_ID==25030]

In [ ]:
patients = pd.read_csv('/Users/grace/mimic/csv/PATIENTS.csv', sep=',')
patients.head()

In [ ]:
patients[patients.SUBJECT_ID==25030]